In [2]:
import os
import pandas as pd

In [555]:
#os.rename(r'C:\Users\nprol195\Downloads\PL Fixtures.csv',r'C:\Users\nprol195\Documents\02 - Weekly\07 - Preppin data\PL Fixtures.csv',)

In [72]:
df = pd.read_csv("PL Fixtures.csv")

In [73]:
#Clean column names
df.columns = [x.lower().strip().replace(' ','_') for x in df.columns]

In [74]:
df = df.dropna().copy()#remove future dates when no matches were played
df.reset_index(drop = True,inplace=True)#Index funky. This assists with future code

In [82]:
#Read instructions in preppin data. Challenge asks for 2 outputs. Output 2 is recalculated without bg six teams
out1 = df.copy()
out1.head()

,round_number,date,location,home_team,away_team,result
0,1,12/09/2020 12:30,Craven Cottage,Fulham,Arsenal,0 - 3
1,1,12/09/2020 15:00,Selhurst Park,Crystal Palace,Southampton,1 - 0
2,1,12/09/2020 17:30,Anfield,Liverpool,Leeds,4 - 3
3,1,12/09/2020 20:00,London Stadium,West Ham,Newcastle,0 - 2
4,1,13/09/2020 14:00,The Hawthorns,West Brom,Leicester,0 - 3


In [88]:
#Remove big six, save as out2
bigsix =  ['Arsenal', 'Chelsea', 'Liverpool', 'Man Utd', 'Man City', 'Spurs']
out2 = out1[(~out1['home_team'].isin(bigsix))&(~out1['away_team'].isin(bigsix))].reset_index(drop=True)

In [90]:
for df in [out1,out2]: #loop through tables
    df[['home','away']] = df['result'].apply(lambda x : pd.Series(str(x).split('-'))) #Splits results clm onto two clm
    df[['home','away']] = df[['home','away']].astype('int') #Converts clms to integers

In [91]:
for df in [out1,out2]: #Loop
    df['homewinlose'] = [3 if df['home'][x] > df['away'][x] else 1 if df['home'][x] == df['away'][x] else 0 for x in  range(df.shape[0])]
    df['awaywinlose'] = [3 if df['home'][x] < df['away'][x] else 1 if df['home'][x] == df['away'][x] else 0 for x in  range(df.shape[0])]
    df['homegoaldiff'] = df['home'] - df['away']
    df['awaygoaldiff'] = df['away'] - df['home']
# expression_if_true if condition else expression_if_false

In [92]:
for df in [out1,out2]:
    df.drop(columns=['result','home','result','away'],inplace=True)

In [93]:
for df in [out1,out2]:
    df['home_team_score'] =  [df['home_team'][x]+'_'+ str(df['homewinlose'][x])+'_'+ str(df['homegoaldiff'][x]) for x in range(df.shape[0])]
    df['away_team_score'] =  [df['away_team'][x]+'_'+ str(df['awaywinlose'][x])+'_'+ str(df['awaygoaldiff'][x]) for x in range(df.shape[0])]


In [94]:
for df in [out1,out2]:
    df.drop(columns=['home_team', 'away_team','homewinlose', 'awaywinlose','homegoaldiff', 'awaygoaldiff'],inplace=True)

In [95]:
out1 = out1.melt(id_vars=['round_number', 'date', 'location'],value_vars=['home_team_score','away_team_score'])
out2 = out2.melt(id_vars=['round_number', 'date', 'location'],value_vars=['home_team_score','away_team_score'])

In [96]:
for df in [out1,out2]:
    df['team_name'] = df['value'].apply(lambda x:x.split('_')[0])
    df['score'] = df['value'].apply(lambda x:x.split('_')[1]).astype('int')
    df['goal_difference'] = df['value'].apply(lambda x:x.split('_')[2]).astype('int')
    df.drop(columns=['variable','value','round_number','date'],inplace=True)

In [97]:
out1gamesplayed = out1.groupby(['team_name']).count()[['location']]
out1gamesscores = out1.groupby(['team_name'])[['score','goal_difference']].sum()

out2gamesplayed = out2.groupby(['team_name']).count()[['location']]
out2gamesscores = out2.groupby(['team_name'])[['score','goal_difference']].sum()


In [98]:
out1= out1gamesplayed.merge(out1gamesscores,on='team_name')
out2= out2gamesplayed.merge(out2gamesscores,on='team_name')

In [99]:
for df in [out1,out2]:
    df.sort_values(by='score',ascending=False,inplace=True)
    df['rank'] = df['score'].rank(ascending=False)
    df.reset_index(inplace=True)

In [100]:
out1 = out1[['rank','team_name','location','score','goal_difference']]
out2 = out2[['rank','team_name','location','score','goal_difference']]

In [101]:
out1.columns = ['position', 'team', 'total_games_played', 'total_points', 'goal_difference']
out2.columns = ['position', 'team', 'total_games_played', 'total_points', 'goal_difference']

In [102]:
out1.head()

,position,team,total_games_played,total_points,goal_difference
0,1.0,Man City,32,74,44
1,2.0,Man Utd,32,66,29
2,3.0,Leicester,31,56,18
3,4.0,West Ham,32,55,11
4,5.0,Chelsea,31,54,19


In [103]:
out1

,position,team,total_games_played,total_points,goal_difference
0,1.0,Man City,32,74,44
1,2.0,Man Utd,32,66,29
2,3.0,Leicester,31,56,18
3,4.0,West Ham,32,55,11
4,5.0,Chelsea,31,54,19
5,6.0,Liverpool,32,53,16
6,7.0,Spurs,32,50,17
7,8.0,Everton,31,49,3
8,9.5,Arsenal,32,46,8
9,9.5,Leeds,32,46,0


In [104]:
out2

,position,team,total_games_played,total_points,goal_difference
0,1.0,West Ham,21,49,21
1,2.0,Leicester,22,40,15
2,3.0,Leeds,22,39,11
3,4.5,Aston Villa,22,34,7
4,4.5,Everton,21,34,4
5,6.5,Crystal Palace,22,32,0
6,6.5,Newcastle,23,32,-4
7,8.5,Southampton,21,30,1
8,8.5,Wolves,22,30,-1
9,10.5,Brighton,22,26,0
